In [14]:
df <- data.frame(matrix(ncol = 3, nrow = 0)) 
colnames(df) <- c("name", "age", "gender")

In [25]:
x <- list(c(1,2))
x1 <- rep(x, 4)

x2 <- list(c(1,2), c(1,2), c(1,2), c(1,2))

x1

[[1]]
[1] 1 2

[[2]]
[1] 1 2

[[3]]
[1] 1 2

[[4]]
[1] 1 2

# DES

In [ ]:
source('Classes.R')
sum_of_squares<-function(point){
   sum <- 0.0        
   for(p in point)
   {
     sum <- sum + p^2
   }
   return(sum)
}

sampleFromHistory <- function(history,historySample,lambda){
  ret <- c()
  for(i in 1:lambda)
    ret <- c(ret,sample(1:ncol(history[[historySample[i]]]), 1))
  return(ret)
}

In [ ]:
point_dim <- 3
N <- point_dim
lambda <- 50
mu <- floor(lambda/2)
cc <- mu/(mu+2)

weights         <- log(mu+1) - log(1:mu)
weights         <- weights/sum(weights)
weightsPop      <- log(lambda+1) - log(1:lambda)
weightsPop      <- weightsPop/sum(weightsPop)  

iter        <- 0L

initFt      <- 1
Ft          <- initFt

history     <- list() 
histHead    <- 0  
histSize <- ceiling(6+ceiling(3*sqrt(N)))
histHead  <- (histHead %% histSize) + 1
histNorm  <- 1/sqrt(2)

FtHistory   <- array(0, histSize)

dMean       <- array(0, dim=c(N,histSize))
pc       <- array(0, dim=c(point_dim,histSize))

objective_fun <- ObjectiveFunction$new(fun=sum_of_squares, dim=point_dim)


gen_random_population <- function(size=lambda, scaler=1){
    coordinates <- scaler * replicate(size, runif(n = point_dim, min = -1, max = 1))
    fitness <- c()
    for(i in 1:dim(coordinates)[2]){
        fitness[i]<-objective_fun$evaluate(coordinates[,i])
    }
    population <- data.frame(t(coordinates), fitness)
    return(population)
}

In [ ]:
population <- gen_random_population()

In [ ]:
matPopNoFitness <- t(data.matrix(population[,!(names(population)=='fitness')]))
popMean <- drop(matPopNoFitness %*% weightsPop)

In [ ]:
population <- population[order(population['fitness'], decreasing=FALSE),]
selectedPoints <- population[1:mu, !(names(population) == 'fitness')]
selectedPoints

In [ ]:
histHead  <- (histHead %% histSize) + 1

history[[histHead]] <- array(0,dim=c(N,mu))
history[[histHead]] <- selectedPoints * histNorm/Ft

In [ ]:
oldMean <- numeric(N)
matSelectedPoints <- t(data.matrix(selectedPoints))
newMean <- drop(matSelectedPoints %*% weights)
newMean

In [ ]:
FtHistory[histHead] = Ft
oldFt <- Ft

In [ ]:
muMean <- newMean
dMean[,histHead] <- (muMean - popMean) / Ft
dMean

In [ ]:
step <- (newMean - oldMean) / Ft
step

In [ ]:
cp <- 1/sqrt(point_dim)
pc[,histHead] = (1 - cp)* rep(0.0, N)/sqrt(N) + sqrt(mu * cp * (2-cp))* step
pc

In [ ]:
limit <- ifelse(iter < histSize, histHead, histSize)
historySample <- sample(1:limit,lambda, T)
historySample2 <- sample(1:limit,lambda, T)

In [ ]:
history

In [ ]:
x1sample <- sampleFromHistory(history,historySample,lambda)
x2sample <- sampleFromHistory(history,historySample,lambda)

In [ ]:
## Make diffs
for (i in 1:lambda) {
    print(i)
    x1 <- history[[historySample[i]]][,x1sample[i]]
    x2 <- history[[historySample[i]]][,x2sample[i]]

    diffs[,i] <- sqrt(cc)*( (x1 - x2) + rnorm(1)*dMean[,historySample[i]] ) + sqrt(1-cc) * rnorm(1)*pc[,historySample2[i]]

}

In [ ]:
dMean

In [ ]:
[historySample[1]]

In [ ]:
history

# DES - kopia

In [1]:
# ARGS
sum_of_squares<-function(point){
   sum <- 0.0        
   for(p in point)
   {
     sum <- sum + p^2
   }
   return(sum)
}

fn <- sum_of_squares
N <- 3

In [2]:
library("ringbuffer")

In [23]:
## Function to check the presence of options in the arguments specified by the user
# @name - argument name
# @default - default value of the argument
# RETURN: value specified by user if the given argument name found, default value otherwise
##
controlParam <- function(name, default) {
v <- control[[name]]
if (is.null(v))
  return (default)
else
  return (v)
}
                                                  ## Number of dimensions
## Check if user specified box constraints values.
## If not accept the default values.
lower <- rep(-100, N)
upper <- rep(100, N)


bounceBackBoundary2 <- function(x){
if(all(x >= cbind(lower)) && all(x <= cbind(upper)))
  return (x)
else if(any(x < cbind(lower)))
  for(i in which(x < cbind(lower)) )
    x[i] <- lower[i] + abs(lower[i] - x[i])%% (upper[i]- lower[i])
  else if(any(x > cbind(upper)))
    for(i in which(x > cbind(upper)) )
      x[i] <- upper[i] - abs(upper[i] - x[i])%% (upper[i]- lower[i])
    x <-deleteInfsNaNs(x)
    return (bounceBackBoundary2(x))
}

sampleFromHistory <- function(history,historySample,lambda){
  ret <- c()
  for(i in 1:lambda)
    ret <- c(ret,sample(1:ncol(history[[historySample[i]]]), 1))
  return(ret)
}

deleteInfsNaNs <- function(x){
  x[is.na(x)] <- .Machine$double.xmax
  x[is.infinite(x)] <- .Machine$double.xmax
  return(x)
}

In [24]:
Ft          <- 1                                ## Scaling factor of difference vectors (a variable!)
initFt      <- 1
stopfitness <-  -Inf                    ## Fitness value after which the convergence is reached
## Strategy parameter setting:
budget      <- 10000*N                     ## The maximum number of fitness function calls
initlambda  <- 4*N       		      ## Population starting size
minlambda   <- 4*N                       ## Population ending size
lambda      <- initlambda                                           ## Population size
mu          <- floor(lambda/2)                  ## Selection size
weights     <- log(mu+1) - log(1:mu)       ## Weights to calculate mean from selected individuals
weights     <- weights/sum(weights)                                 ##    \-> weights are normalized by the sum
weightsSumS <- sum(weights^2)                                       ## weights sum square
mueff       <- sum(weights)^2/sum(weights^2) ## Variance effectiveness factor
cc          <- mu/(mu+2)                      ## Evolution Path decay factor
pathLength  <- 6                       ## Size of evolution path
cp          <- 1/sqrt(N)                        ## Evolution Path decay factor
maxiter     <- floor(budget/(lambda+1))      ## Maximum number of iterations after which algorithm stops
c_Ft        <- 0
pathRatio   <- sqrt(pathLength)           ## Path Length Control reference value
histSize    <- ceiling(6+ceiling(3*sqrt(N)))## Size of the window of history - the step length history
Ft_scale    <- ((mueff+2)/(N+mueff+3))/(1 + 2*max(0, sqrt((mueff-1)/(N+1))-1) + (mueff+2)/(N+mueff+3))
tol         <- 10^-6
counteval   <- 0                                                    ## Number of function evaluations
sqrt_N      <- sqrt(N)

In [25]:
Lamarckism <- FALSE

## Fitness function wrapper
fn_ <- function(x){
if(all(x >= cbind(lower)) && all(x <= cbind(upper))){
  counteval <<- counteval + 1
  return (fn(x))
}
else
  return(.Machine$double.xmax)
}

## Fitness function wrapper for Lamarcian approach
fn_l <- function(P){
if(is.matrix(P)){
  if(counteval + ncol(P) <= budget)
    return ( apply(P, 2, fn_) )
  else{
    ret <- c()
    budLeft <- budget-counteval
    for (i in 1:budLeft ) {
      ret <- c(ret,fn_(P[,i]))
    }
    return(c(ret,rep(.Machine$double.xmax,ncol(P)-budLeft)))
  }
}else{
  if(counteval < budget)
    return ( fn_(P) )
  else
    return(.Machine$double.xmax)
}
}

## Fitness function wrapper for nonLamarckian approach
fn_d <- function(P, P_repaired, fitness) {
P <- deleteInfsNaNs(P)
P_repaired <- deleteInfsNaNs(P_repaired)

if(is.matrix(P) && is.matrix(P_repaired)){
  repairedInd <- apply(P!=P_repaired,2,all)
  P_fit <- fitness
  vecDist <- colSums((P - P_repaired)^2)
  P_fit[which(repairedInd)] <- worst.fit + vecDist[which(repairedInd)]
  P_fit <- deleteInfsNaNs(P_fit)
  return(P_fit)
}else{
  P_fit <- fitness
  if (P!=P_repaired){
    P_fit <- worst.fit + (sum(P-P_repaired)^2)
    P_fit <- deleteInfsNaNs(P_fit)
  }
  return (P_fit)
}

}

In [26]:
best.fit        <- Inf                  ## The best fitness found so far
best.par        <- NULL                 ## The best solution found so far
worst.fit       <- NULL                 ## The worst solution found so far:
last.restart    <-0
restart.length  <-0
restart.number  <-0

In [27]:
steps       <- ringbuffer(size = pathLength*N)                      ## Cyclical buffer containing last 'pathLength' steps of algorithm
dMean       <- array(0, dim=c(N,histSize))
FtHistory   <- array(0, histSize)                                   ## Array buffer containing 'histSize' last values of 'Ft'
pc       <- array(0, dim=c(N,histSize))

## Initialize internal strategy parameters
msg             <- NULL                                             ## Reason for terminating
restart.number  <- -1

In [28]:
lambda          <- 50
mu              <- floor(lambda/2)
weights         <- log(mu+1) - log(1:mu)
weights         <- weights/sum(weights)
weightsPop      <- log(lambda+1) - log(1:lambda)
weightsPop      <- weightsPop/sum(weightsPop)

histHead    <- 0                                                      ## Pointer to the history buffer head
iter        <- 0L                                                     ## Number of iterations
history     <- list()                                                 ## List stores best 'mu'(variable) individuals for 'hsize' recent iterations
Ft          <- initFt

# Create fisrt population
population <- replicate(lambda, runif(N,0.8*lower,0.8*upper))
cumMean=(upper+lower)/2
# populationRepaired <- apply(population,2,bounceBackBoundary2)

# if(Lamarckism==TRUE){
#   population <- populationRepaired
# }

selection       <- rep(0, mu)
selectedPoints  <- matrix(0, nrow=N, ncol=mu)
fitness         <- fn_l(population)
oldMean         <- numeric(N)
newMean         <- c(10,10,10)
limit           <- 0
worst.fit       <- max(fitness)

# Store population and selection means
popMean         <- drop(population %*% weightsPop)
muMean          <- newMean

## Matrices for creating diffs
diffs     <- matrix(0, N, lambda)
x1sample  <- numeric(lambda)
x2sample  <- numeric(lambda)

chiN      <- (sqrt(2)*gamma((N+1)/2)/gamma(N/2))
histNorm  <- 1/sqrt(2)
counterRepaired <- 0

stoptol=F

In [29]:
# WHILE starts

In [30]:
population

-13.17548,-53.10392,76.27985,11.357417,-4.670647,-27.909800,2.390076,-14.56116,-24.01764,39.86708,...,14.87888,19.30104,-14.404668,-68.72108,30.699182,54.18268,10.69644,30.94558,-66.18250,9.962519
62.94346,-43.31482,40.37250,46.078297,-25.178938,-3.659226,77.713198,31.51500,66.09670,20.74519,...,50.15714,76.09537,7.581243,70.44468,7.404058,-65.52662,-78.31497,23.39124,-55.02284,-34.522131
-69.18472,18.34165,-22.74644,-6.133448,38.207383,-51.931039,-52.539052,-18.19076,41.08242,-68.10708,...,-22.72004,51.60320,50.932125,-21.35658,-10.604431,-18.29960,-62.73741,57.13145,-52.05034,-69.256650


In [31]:
iter      <- iter + 1L
histHead  <- (histHead %% histSize) + 1

lambda <- lambda
#lambda      <- round(((minlambda-initlambda)/budget)*counteval+initlambda)
mu          <- floor(lambda/2)
weights <- log(mu+1) - log(1:mu)
weights <- weights/sum(weights)

selection       <- order(fitness)[1:mu]
selectedPoints  <- population[,selection]

# Save selected population in the history buffer
history[[histHead]] <- array(0,dim=c(N,mu))
history[[histHead]] <- selectedPoints * histNorm/Ft

## Calculate weighted mean of selected points
oldMean <- newMean
newMean <- drop(selectedPoints %*% weights)

## Write to buffers
muMean <- newMean
dMean[,histHead] <- (muMean - popMean) / Ft

step <- (newMean - oldMean) / Ft

## Update Ft
FtHistory[histHead] = Ft
oldFt <- Ft

In [32]:
if(histHead==1){
pc[,histHead] = (1 - cp)* rep(0.0, N)/sqrt(N) + sqrt(mu * cp * (2-cp))* step
}else{
pc[,histHead] = (1 - cp)* pc[,histHead-1] + sqrt(mu * cp * (2-cp))* step
}
## Sample from history with uniform distribution
limit <- ifelse(iter < histSize, histHead, histSize)
historySample <- sample(1:limit,lambda, T)
historySample2 <- sample(1:limit,lambda, T)

x1sample <- sampleFromHistory(history,historySample,lambda)
x2sample <- sampleFromHistory(history,historySample,lambda)

## Make diffs
for (i in 1:lambda) {
x1 <- history[[historySample[i]]][,x1sample[i]]
x2 <- history[[historySample[i]]][,x2sample[i]]

diffs[,i] <- sqrt(cc)*( (x1 - x2) + rnorm(1)*dMean[,historySample[i]] ) + sqrt(1-cc) * rnorm(1)*pc[,historySample2[i]]

}

## New population
population <- newMean + Ft * diffs + tol*rnorm(diffs)/chiN
population <- deleteInfsNaNs(population)

# Check constraints violations
# Repair the individual if necessary
populationTemp <- population
populationRepaired <- apply(population,2,bounceBackBoundary2)

counterRepaired=0
for(tt in 1:ncol(populationTemp)){
if(any(populationTemp[,tt] != populationRepaired[,tt]))
  counterRepaired = counterRepaired + 1
}

if(Lamarckism==TRUE){
population <- populationRepaired
}

popMean <- drop(population %*% weightsPop)

## Evaluation
fitness <- fn_l(population)
if(Lamarckism==FALSE){
fitnessNonLamarcian <- fn_d(population, populationRepaired, fitness)
}

## Break if fit :
wb <- which.min(fitness)
if (fitness[wb] < best.fit) {
best.fit <- fitness[wb]
if(Lamarckism==TRUE)
  best.par <- population[,wb]
else
  best.par <- populationRepaired[,wb]
}

## Check worst fit:
ww <- which.max(fitness)
if (fitness[ww] > worst.fit){
worst.fit <- fitness[ww]
}

## Fitness with penalty for nonLamarcian approach
if(Lamarckism==FALSE){
fitness <- fitnessNonLamarcian
}


## Check if the middle point is the best found so far
cumMean <- 0.8*cumMean+0.2*newMean
cumMeanRepaired <-bounceBackBoundary2(cumMean)

fn_cum  <- fn_l(cumMeanRepaired)
if (fn_cum < best.fit) {
best.fit <- drop(fn_cum)
best.par <- cumMeanRepaired
}

## Escape from flat-land:
#if (min(fitness) == sort(fitness,partial=min(1+floor(lambda/2), 2+ceiling(lambda/4)))[min(1+floor(lambda/2), 2+ceiling(lambda/4))]) {
#  Ft <- Ft * exp(0.2*Ft_scale);
#}

if (fitness[1] <= stopfitness) {
msg <- "Stop fitness reached."
break
}

if(abs(range(fitness)[2] - range(fitness)[1]) < tol)
{
if (counteval < 0.8*budget)
  stoptol=T
}

In [34]:
population

17.51110,-28.998623,29.097916,-14.08666,-56.75207,64.030123,-67.625241,39.45370,22.24808,3.6319419,...,-19.807510,47.06330,-39.80114,26.3885760,-18.45710,-12.825907,58.28964,-28.86170,-17.690594,-13.79157
19.60804,1.986796,-35.179922,-16.47040,-12.32426,33.620889,-2.635209,88.50919,16.72493,23.9157890,...,79.458792,13.88332,-49.90476,-0.8790023,-20.02077,-2.771494,13.46975,-7.82110,-42.307188,38.15609
73.98926,44.037662,1.697442,-26.67918,-86.82457,3.165437,21.669500,78.75957,31.14066,-0.6469578,...,6.600824,-14.99615,18.85075,-34.1762635,-18.49580,-56.113205,-45.70738,-53.52491,7.694993,15.54533


In [33]:
cnt <- c(`function`=as.integer(counteval))

## Drop names from value object
names(best.fit) <- NULL
res <- list(par=best.par,
          value=best.fit,
          counts=cnt,
          resets=restart.number,
          convergence=ifelse(iter >= maxiter, 1L, 0L),
          message=msg,
          diagnostic=log
)
class(res) <- "cmade.result"

return(res)

$par
[1] 0.33735182 0.14577276 0.04698772

$value
[1] 0.1372638

$counts
function 
     101 

$resets
[1] -1

$convergence
[1] 0

$message
NULL

$diagnostic
function (x, base = exp(1))  .Primitive("log")

attr(,"class")
[1] "cmade.result"